In [1]:
import pandas as pd
from glob import glob

In [2]:
count = []
arquivos_treino = glob('./files/treino/*.csv')
for arquivo in arquivos_treino:
    df = pd.read_csv(arquivo)
    count.append(df.count())
del(df)

print(pd.DataFrame(count).T)

                              0       1       2       3       4      5
userId                   100000  100000  100000  100000  100000  77942
userType                 100000  100000  100000  100000  100000  77942
historySize              100000  100000  100000  100000  100000  77942
history                  100000  100000  100000  100000  100000  77942
timestampHistory         100000  100000  100000  100000  100000  77942
numberOfClicksHistory    100000  100000  100000  100000  100000  77942
timeOnPageHistory        100000  100000  100000  100000  100000  77942
scrollPercentageHistory  100000  100000  100000  100000  100000  77942
pageVisitsCountHistory   100000  100000  100000  100000  100000  77942
timestampHistory_new     100000  100000  100000  100000  100000  77942


In [3]:
count = []
arquivos_itens = glob('./itens/itens/*.csv')
for arquivo in arquivos_itens:
    df = pd.read_csv(arquivo)
    count.append(df.count())
del(df)

print(pd.DataFrame(count).T)

               0       1      2
page      100000  100000  55603
url       100000  100000  55603
issued    100000  100000  55603
modified  100000  100000  55603
title     100000  100000  55603
body      100000  100000  55603
caption   100000  100000  55603


In [4]:
treino_df = pd.concat([pd.read_csv(arquivo) for arquivo in arquivos_treino], ignore_index=True)
itens_df = pd.concat([pd.read_csv(arquivo) for arquivo in arquivos_itens], ignore_index=True)

In [5]:
treino_df.isnull().sum()

userId                     0
userType                   0
historySize                0
history                    0
timestampHistory           0
numberOfClicksHistory      0
timeOnPageHistory          0
scrollPercentageHistory    0
pageVisitsCountHistory     0
timestampHistory_new       0
dtype: int64

In [6]:
itens_df.isnull().sum()

page        0
url         0
issued      0
modified    0
title       0
body        0
caption     0
dtype: int64

In [7]:
treino_df.duplicated().sum()

np.int64(0)

In [8]:
itens_df.duplicated().sum()

np.int64(0)

In [9]:
treino_df.head()

,userId,userType,historySize,history,timestampHistory,numberOfClicksHistory,timeOnPageHistory,scrollPercentageHistory,pageVisitsCountHistory,timestampHistory_new
0,f98d1132f60d46883ce49583257104d15ce723b3bbda21...,Non-Logged,3,"c8aab885-433d-4e46-8066-479f40ba7fb2, 68d2039c...","1657146417045, 1657146605778, 1657146698738","76, 38, 41","20380, 21184, 35438","50.3, 18.18, 16.46","2, 1, 1","1657146417045, 1657146605778, 1657146698738"
1,2c1080975e257ed630e26679edbe4d5c850c65f3e09f65...,Non-Logged,60,"3325b5a1-979a-4cb3-82b6-63905c9edbe8, fe856057...","1656684240278, 1656761266729, 1656761528085, 1...","7, 80, 2, 1, 7, 62, 26, 44, 4, 4, 14, 45, 13, ...","6049, 210489, 8672, 10000, 30000, 123007, 9965...","25.35, 45.66, 35.3, 28.05, 36.53, 47.57, 55.33...","1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 1, 1, 1, 2, 1, 1...","1656684240278, 1656761266729, 1656761528085, 1..."
2,0adffd7450d3b9840d8c6215f0569ad942e782fb19b805...,Logged,107,"04756569-593e-4133-a95a-83d35d43dbbd, 29b6b142...","1656678946256, 1656701076495, 1656701882565, 1...","0, 0, 0, 0, 0, 44, 0, 0, 2, 1, 0, 0, 0, 44, 0,...","311274, 140000, 32515, 157018, 118689, 159243,...","67.58, 47.22, 41.52, 63.09, 51.38, 65.11, 71.9...","1, 1, 1, 1, 1, 1, 1, 1, 2, 1, 1, 1, 1, 1, 1, 1...","1656678946256, 1656701076495, 1656701882565, 1..."
3,c1e8d644329a78ea1f994292db624c57980b2886cfbc2d...,Non-Logged,56,"1f2b9c2f-a2d2-4192-b009-09065da8ec23, 04756569...","1658333312180, 1658404553818, 1658408449062, 1...","8, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 2, 0, 1, 1...","182696, 91925, 30000, 273655, 126409, 42980, 1...","58.26, 72.66, 22.57, 59.89, 40.36, 36.35, 14.7...","1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1...","1658333312180, 1658404553818, 1658408449062, 1..."
4,e777d1f31d4d955b63d60acc13df336d3903f52ab8f8f4...,Non-Logged,4,"bebdeb3e-1699-43e0-a1b8-989f5a6ab679, f4b484a7...","1658766608801, 1658766608801, 1660084035094, 1...","579, 579, 7, 2","801396, 801396, 10000, 10000","78.74, 78.74, 16.71, 9.34","7, 7, 1, 1","1658766608801, 1658766608801, 1660084035094, 1..."


In [10]:
itens_df.head()

,page,url,issued,modified,title,body,caption
0,13db0ab1-eea2-4603-84c4-f40a876c7400,http://g1.globo.com/am/amazonas/noticia/2022/0...,2022-06-18 20:37:45+00:00,2023-04-15 00:02:08+00:00,Caso Bruno e Dom: 3º suspeito tem prisão tempo...,"Após audiência de custódia, a Justiça do Amazo...",Jeferson da Silva Lima foi escoltado por agent...
1,92907b73-5cd3-4184-8d8c-e206aed2bf1c,http://g1.globo.com/pa/santarem-regiao/noticia...,2019-06-20 17:19:52+00:00,2023-06-16 20:19:15+00:00,Linguajar dos santarenos é diferenciado e chei...,Vista aérea de Santarém\nÁdrio Denner/ AD Prod...,As expressões santarenas não significam apenas...
2,61e07f64-cddf-46f2-b50c-ea0a39c22050,http://g1.globo.com/mundo/noticia/2022/07/08/e...,2022-07-08 08:55:52+00:00,2023-04-15 04:25:39+00:00,Ex-premiê Shinzo Abe morre após ser baleado no...,Novo vídeo mostra que assassino de Shinzo Abe ...,Ex-primeiro-ministro foi atingido por tiros de...
3,30e2e6c5-554a-48ed-a35f-6c6691c8ac9b,http://g1.globo.com/politica/noticia/2021/09/0...,2021-09-09 19:06:46+00:00,2023-06-07 17:44:54+00:00,"Relator no STF, Fachin vota contra marco tempo...","Relator no STF, Fachin vota contra marco tempo...",Ministro defendeu que posse indígena é diferen...
4,9dff71eb-b681-40c7-ac8d-68017ac36675,http://g1.globo.com/politica/noticia/2021/09/1...,2021-09-15 19:16:13+00:00,2023-06-07 17:43:39+00:00,"\nApós 2 votos, pedido de vista suspende julga...",Após um pedido de vista (mais tempo para análi...,"Pelo marco temporal, índios só podem reivindic..."


In [12]:
historico = [
    'history',
    'timestampHistory',
    'numberOfClicksHistory',
    'timeOnPageHistory',
    'scrollPercentageHistory',
    'pageVisitsCountHistory',
    'timestampHistory_new'
]

logs_df = treino_df.copy()
for coluna in historico:
    logs_df[coluna] = logs_df[coluna].apply(lambda x: str(x).split(", "))
logs_df = logs_df.explode(historico, ignore_index=True)

In [13]:
logs_df.head()

,userId,userType,historySize,history,timestampHistory,numberOfClicksHistory,timeOnPageHistory,scrollPercentageHistory,pageVisitsCountHistory,timestampHistory_new
0,f98d1132f60d46883ce49583257104d15ce723b3bbda21...,Non-Logged,3,c8aab885-433d-4e46-8066-479f40ba7fb2,1657146417045,76,20380,50.3,2,1657146417045
1,f98d1132f60d46883ce49583257104d15ce723b3bbda21...,Non-Logged,3,68d2039c-c9aa-456c-ac33-9b2e8677fba7,1657146605778,38,21184,18.18,1,1657146605778
2,f98d1132f60d46883ce49583257104d15ce723b3bbda21...,Non-Logged,3,13e423ce-1d69-4c78-bc18-e8c8f7271964,1657146698738,41,35438,16.46,1,1657146698738
3,2c1080975e257ed630e26679edbe4d5c850c65f3e09f65...,Non-Logged,60,3325b5a1-979a-4cb3-82b6-63905c9edbe8,1656684240278,7,6049,25.35,1,1656684240278
4,2c1080975e257ed630e26679edbe4d5c850c65f3e09f65...,Non-Logged,60,fe856057-f97d-419f-ab1c-97c5c3e0719c,1656761266729,80,210489,45.66,1,1656761266729


In [18]:
logs_df.isnull().sum()

userId                     0
userType                   0
historySize                0
history                    0
timestampHistory           0
numberOfClicksHistory      0
timeOnPageHistory          0
scrollPercentageHistory    0
pageVisitsCountHistory     0
timestampHistory_new       0
dtype: int64

In [19]:
logs_df.shape

(8123951, 10)

In [20]:
logs_df = logs_df.drop('historySize', axis=1)

In [24]:
logs_df['history'] = logs_df['history'].astype(str)
itens_df['page'] = itens_df['page'].astype(str)

In [27]:
merged_df = logs_df.merge(itens_df, left_on='history', right_on='page', how='inner')

In [28]:
merged_df.shape

(8123951, 16)

In [ ]:
merged_df = merged_df.sort_values(by=["userId", "timestampHistory"], ascending=[True, False])

In [ ]:
rel_col = []
prev_user = None
inverse_aux = merged_df[::-1].copy()
for i in range(len(merged_df)):
    user = inverse_aux.userId.iloc[i]
    if user != prev_user:
        relevance = 1
    else:
        relevance = relevance + 1
    rel_col.append(relevance)
    prev_user = user
inverse_aux['relevance'] = rel_col
final = inverse_aux[::-1]